In [1]:
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
from qlib.utils import flatten_dict

from qlib.data import D
from pprint import pprint
import yaml

# 读入yaml配置文件，注意文件路径的写法
with open('/home/nbcctwya/Python_Project/qlib/examples/benchmarks/LightGBM/workflow_config_lightgbm_Alpha158.yaml', 'rb') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

pprint(config)

{'benchmark': 'SH000300',
 'data_handler_config': {'end_time': datetime.date(2020, 8, 1),
                         'fit_end_time': datetime.date(2014, 12, 31),
                         'fit_start_time': datetime.date(2008, 1, 1),
                         'instruments': 'csi300',
                         'start_time': datetime.date(2008, 1, 1)},
 'market': 'csi300',
 'port_analysis_config': {'backtest': {'account': 100000000,
                                       'benchmark': 'SH000300',
                                       'end_time': datetime.date(2020, 8, 1),
                                       'exchange_kwargs': {'close_cost': 0.0015,
                                                           'deal_price': 'close',
                                                           'limit_threshold': 0.095,
                                                           'min_cost': 5,
                                                           'open_cost': 0.0005},
                          

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
provider_uri = config['qlib_init']['provider_uri']
qlib.init(provider_uri=provider_uri, region=config['qlib_init']['region'])

[33514:MainThread](2025-12-16 15:33:00,310) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[33514:MainThread](2025-12-16 15:33:00,315) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[33514:MainThread](2025-12-16 15:33:00,316) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': PosixPath('/home/nbcctwya/.qlib/qlib_data/cn_data')}


# 核心工作流
本例中训练、预测、回测三个步骤合并成一个实验
## 配置参数，实例化模型对象和因子数据集对象（含因子和标签）

In [3]:
# 之前的参数配置不需要了，因为已经从yaml文件导入了

# 实例化模型对象
model = init_instance_by_config(config['task']["model"])
dataset = init_instance_by_config(config['task']["dataset"])

ModuleNotFoundError. CatBoostModel are skipped. (optional: maybe installing CatBoostModel can fix it.)
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).
ModuleNotFoundError.  PyTorch models are skipped (optional: maybe installing pytorch can fix it).


[33514:MainThread](2025-12-16 15:57:55,919) INFO - qlib.timer - [log.py:127] - Time cost: 12.876s | Loading data Done
[33514:MainThread](2025-12-16 15:57:56,599) INFO - qlib.timer - [log.py:127] - Time cost: 0.125s | DropnaLabel Done
[33514:MainThread](2025-12-16 15:57:58,939) INFO - qlib.timer - [log.py:127] - Time cost: 2.339s | CSZScoreNorm Done
[33514:MainThread](2025-12-16 15:57:58,941) INFO - qlib.timer - [log.py:127] - Time cost: 3.019s | fit & process data Done
[33514:MainThread](2025-12-16 15:57:58,942) INFO - qlib.timer - [log.py:127] - Time cost: 15.898s | Init data Done


# 执行工作流实验

In [4]:
# R变量可以理解为试验记录管理器
with R.start(experiment_name="workflow"): # 注意设置好实验名
    # 可选：记录task中的参数到运行记录下的params目录
    R.log_params(**flatten_dict(config['task']))

    ############
    # 训练
    ############
    model.fit(dataset)

    # 训练好的模型以pkl文件形式保存到本次实验运行记录目录下的artifacts子目录
    R.save_objects(**{"trained_model.pkl": model})

    ############
    # 预测
    ############
    # 本次实验的实验记录器
    recorder = R.get_recorder()
    # 生成预测结果文件
    sig_rec = SignalRecord(model, dataset, recorder)
    sig_rec.generate()

    # 生成预测结果分析文件
    sigAna_rec = SigAnaRecord(recorder)
    sigAna_rec.generate()

    ############
    # 回测
    ############

    # 策略model和dataset参数要重新设置，指向对应实例变量
    config["port_analysis_config"]['strategy']['kwargs']['model'] = model
    config["port_analysis_config"]['strategy']['kwargs']['dataset'] = dataset

    pa_rec = PortAnaRecord(recorder, config["port_analysis_config"], "day")
    pa_rec.generate()

    # 打印本次实验记录器信息，含记录器id，experiment_id等信息
    print('info', R.get_recorder().info)

[33514:MainThread](2025-12-16 16:07:36,743) INFO - qlib.workflow - [exp.py:258] - Experiment 457878458179041383 starts running ...
[33514:MainThread](2025-12-16 16:07:36,827) INFO - qlib.workflow - [recorder.py:345] - Recorder 22d00da9fc5a476b834c6571b178ed80 starts running under Experiment 457878458179041383 ...
usage: git diff --no-index [<options>] <path> <path>

Diff output format options
    -p, --patch           generate patch
    -s, --no-patch        suppress diff output
    -u                    generate patch
    -U, --unified[=<n>]   generate diffs with <n> lines context
    -W, --function-context
                          generate diffs with <n> lines context
    --raw                 generate the diff in raw format
    --patch-with-raw      synonym for '-p --raw'
    --patch-with-stat     synonym for '-p --stat'
    --numstat             machine friendly --stat
    --shortstat           output only the last line of --stat
    -X, --dirstat[=<param1,param2>...]
            

Training until validation scores don't improve for 50 rounds
[20]	train's l2: 0.981377	valid's l2: 0.993566
[40]	train's l2: 0.973811	valid's l2: 0.993977
[60]	train's l2: 0.967152	valid's l2: 0.994648
Early stopping, best iteration is:
[18]	train's l2: 0.982182	valid's l2: 0.993497


[33514:MainThread](2025-12-16 16:07:54,006) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 457878458179041383


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2017-01-03 SH600000   -0.027139
           SH600008    0.007009
           SH600009    0.009770
           SH600010    0.012275
           SH600015   -0.112909


{'IC': np.float64(0.04680587323833807),
 'ICIR': np.float64(0.3815683918932706),
 'Rank IC': np.float64(0.049049290457489736),
 'Rank ICIR': np.float64(0.406748756941287)}


...

backtest loop:   0%|          | 0/871 [00:00<?, ?it/s]

/home/nbcctwya/anaconda3/envs/qlib_experiment/lib/python3.9/site-packages/qlib/utils/index_data.py:492: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
/home/nbcctwya/anaconda3/envs/qlib_experiment/lib/python3.9/site-packages/qlib/utils/index_data.py:492: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
/home/nbcctwya/anaconda3/envs/qlib_experiment/lib/python3.9/site-packages/qlib/utils/index_data.py:492: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
[33514:MainThread](2025-12-16 16:08:21,995) INFO - qlib.workflow - [record_temp.py:515] - Portfolio analysis record 'port_analysis_1day.pkl' has been saved as the artifact of the Experiment 457878458179041383
[33514:MainThread](2025-12-16 16:08:22,006) INFO - qlib.workflow - [record_temp.py:540] - Indicator analysis record 'indicator_analysis_1day.pkl' has been saved as the artifact of the Experiment 457878458179041383


'The following are analysis results of benchmark return(1day).'
                       risk
mean               0.000477
std                0.012295
annualized_return  0.113561
information_ratio  0.598699
max_drawdown      -0.370479
'The following are analysis results of the excess return without cost(1day).'
                       risk
mean               0.000530
std                0.005718
annualized_return  0.126029
information_ratio  1.428574
max_drawdown      -0.072310
'The following are analysis results of the excess return with cost(1day).'
                       risk
mean               0.000339
std                0.005717
annualized_return  0.080654
information_ratio  0.914486
max_drawdown      -0.086083
'The following are analysis results of indicators(1day).'
     value
ffr    1.0
pa     0.0
pos    0.0


[33514:MainThread](2025-12-16 16:08:22,609) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


info {'class': 'Recorder', 'id': '22d00da9fc5a476b834c6571b178ed80', 'name': 'mlflow_recorder', 'experiment_id': '457878458179041383', 'start_time': '2025-12-16 16:07:36', 'end_time': None, 'status': 'RUNNING'}


# 预测结果查询
### 标签值label和预测值score

In [5]:
# label_df = predict_recorder.load_object("label.pkl") # 这个pkl文件记录的是测试集未经数据预处理的原始标签值
label_df = dataset.prepare("test", col_set="label") # 测试集标签值，默认这是经过数据预处理的标签
label_df.columns = ['label'] # 修改列名LABEL0为label

pred_df = recorder.load_object("pred.pkl") # 加载测试集预测结果到dataframe

print('label_df', label_df) # 预处理后的测试集标签值
print('pred_df', pred_df) # 测试集对标签的预测值，score就是预测值

label_df                           label
datetime   instrument          
2017-01-03 SH600000   -0.001831
           SH600008   -0.002398
           SH600009    0.001493
           SH600010    0.003520
           SH600015   -0.007142
...                         ...
2020-07-31 SZ300413   -0.037566
           SZ300433   -0.031677
           SZ300498   -0.006531
           SZ300601    0.090264
           SZ300628    0.004142

[261300 rows x 1 columns]
pred_df                           score
datetime   instrument          
2017-01-03 SH600000   -0.027139
           SH600008    0.007009
           SH600009    0.009770
           SH600010    0.012275
           SH600015   -0.112909
...                         ...
2020-07-31 SZ300413   -0.018151
           SZ300433   -0.100623
           SZ300498   -0.040878
           SZ300601   -0.190996
           SZ300628    0.044844

[261300 rows x 1 columns]
